In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
KEY = os.getenv("GEMINI_KEY")
if not KEY:
    raise ValueError("GEMINI_KEY not set in environment variables.")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  
    google_api_key=KEY,
    convert_system_message_to_human=True  
)

In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import  SequentialChain
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_gen_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [11]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_gen_prompt,output_key="quiz",verbose=True)

C:\Users\shakt\AppData\Local\Temp\ipykernel_28852\4197540469.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm=llm,prompt=quiz_gen_prompt,output_key="quiz",verbose=True)


In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [14]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [17]:
with open("D:\MCQgen\data.txt","r") as file:
    TEXT = file.read()

In [19]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [21]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\shakt\AppData\Local\Temp\ipykernel_28852\1882050128.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(
d:\MCQgen\env\lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

d:\MCQgen\env\lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



> Finished chain.

> Finished chain.


In [22]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [ ]:
quiz = response.get("quiz")

print("RAW QUIZ OUTPUT:")
print(quiz if quiz else "Quiz not found in response.")


RAW QUIZ OUTPUT:
```json
{
"1": {
"mcq": "Which of the following is NOT a unifying theme in biology?",
"options": {
"a": "All organisms are made of cells.",
"b": "Evolution explains the diversity of life.",
"c": "Organisms can regulate their internal environments.",
"d": "Organisms are always getting bigger."},
"correct": "d"
},
"2": {
"mcq": "Biologists study life at different levels of organization. Which of these is an example of a level they study?",
"options": {
"a": "The molecular biology of a cell.",
"b": "The history of the universe.",
"c": "The chemical composition of rocks.",
"d": "The stock market."},
"correct": "a"
},
"3": {
"mcq": "What do biologists use to make observations, generate hypotheses and perform experiments?",
"options": {
"a": "The scientific method.",
"b": "Pure luck.",
"c": "Their feelings.",
"d": "Ancient texts."},
"correct": "a"
},
"4": {
"mcq": "Approximately how long ago did life emerge on Earth?",
"options": {
"a": "3.7 million years ago.",
"b": "3.7 bi

In [35]:
import re
import json

def clean_json(text):
    if not text:
        return None
    text = text.strip()
    text = re.sub(r"^```json|```$", "", text)  # remove code fences
    text = re.sub(r"^.*?{", "{", text, flags=re.DOTALL)  # remove leading text before first `{`
    return text

quiz_cleaned = clean_json(quiz)

try:
    quiz_dict = json.loads(quiz_cleaned)
except json.JSONDecodeError as e:
    quiz_dict = None


In [36]:
print(json.dumps(quiz_dict, indent=2))


{
  "1": {
    "mcq": "Which of the following is NOT a unifying theme in biology?",
    "options": {
      "a": "All organisms are made of cells.",
      "b": "Evolution explains the diversity of life.",
      "c": "Organisms can regulate their internal environments.",
      "d": "Organisms are always getting bigger."
    },
    "correct": "d"
  },
  "2": {
    "mcq": "Biologists study life at different levels of organization. Which of these is an example of a level they study?",
    "options": {
      "a": "The molecular biology of a cell.",
      "b": "The history of the universe.",
      "c": "The chemical composition of rocks.",
      "d": "The stock market."
    },
    "correct": "a"
  },
  "3": {
    "mcq": "What do biologists use to make observations, generate hypotheses and perform experiments?",
    "options": {
      "a": "The scientific method.",
      "b": "Pure luck.",
      "c": "Their feelings.",
      "d": "Ancient texts."
    },
    "correct": "a"
  },
  "4": {
    "mc

In [37]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [38]:
quiz_table_data

[{'MCQ': 'Which of the following is NOT a unifying theme in biology?',
  'Choices': 'a: All organisms are made of cells. | b: Evolution explains the diversity of life. | c: Organisms can regulate their internal environments. | d: Organisms are always getting bigger.',
  'Correct': 'd'},
 {'MCQ': 'Biologists study life at different levels of organization. Which of these is an example of a level they study?',
  'Choices': 'a: The molecular biology of a cell. | b: The history of the universe. | c: The chemical composition of rocks. | d: The stock market.',
  'Correct': 'a'},
 {'MCQ': 'What do biologists use to make observations, generate hypotheses and perform experiments?',
  'Choices': 'a: The scientific method. | b: Pure luck. | c: Their feelings. | d: Ancient texts.',
  'Correct': 'a'},
 {'MCQ': 'Approximately how long ago did life emerge on Earth?',
  'Choices': 'a: 3.7 million years ago. | b: 3.7 billion years ago. | c: 37,000 years ago. | d: 370 years ago.',
  'Correct': 'b'},
 {'M

In [40]:
quiz = pd.DataFrame(quiz_table_data)

In [41]:
quiz.to_csv("quiz.csv",index=False)